<a href="https://colab.research.google.com/github/alexjohnson21/ubiquitous-sniffle/blob/master/cse450_prove04_part03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn import tree
from sklearn.model_selection import KFold

from google.colab import files
#upload = files.upload()

In [2]:
data = pd.read_csv('crx.data', na_values=['?'], header=None, skipinitialspace=True)
data.columns = ['A1','A2','A3','A4','A5','A6','A7','A8','A9','A10','A11','A12','A13','A14','A15','A16']

data.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+


In [3]:
data.isna().sum()

A1     12
A2     12
A3      0
A4      6
A5      6
A6      9
A7      9
A8      0
A9      0
A10     0
A11     0
A12     0
A13     0
A14    13
A15     0
A16     0
dtype: int64

In [4]:
imputer_continuous = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer_discrete = SimpleImputer(missing_values= np.nan, strategy= 'most_frequent')

data[['A2', 'A14']] = imputer_continuous.fit_transform(data[['A2', 'A14']])
data[['A1', 'A4', 'A5', 'A6', 'A7']] = imputer_discrete.fit_transform(data[['A1', 'A4', 'A5', 'A6', 'A7']])

data.isna().sum()
data

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,202.0,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,43.0,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,280.0,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,100.0,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,120.0,0,+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
685,b,21.08,10.085,y,p,e,h,1.25,f,f,0,f,g,260.0,0,-
686,a,22.67,0.750,u,g,c,v,2.00,f,t,2,t,g,200.0,394,-
687,a,25.25,13.500,y,p,ff,ff,2.00,f,t,1,t,g,200.0,1,-
688,b,17.92,0.205,u,g,aa,v,0.04,f,f,0,f,g,280.0,750,-


## Decision Tree with One-Hot Encoding on non-numerics
This admittedly produces a large number of columns that are difficult to read, but should be efficient for training purposes

In [5]:
X = data.iloc[:, :-1]
y = data.iloc[:, -1].map({"+":1, "-":0})

X.head()
y.head()

0    1
1    1
2    1
3    1
4    1
Name: A16, dtype: int64

In [6]:
X = pd.get_dummies(X, columns=X.select_dtypes('object').columns)
print(X.head())

X.to_numpy()

      A2     A3    A8  A11    A14  ...  A12_f  A12_t  A13_g  A13_p  A13_s
0  30.83  0.000  1.25    1  202.0  ...      1      0      1      0      0
1  58.67  4.460  3.04    6   43.0  ...      1      0      1      0      0
2  24.50  0.500  1.50    0  280.0  ...      1      0      1      0      0
3  27.83  1.540  3.75    5  100.0  ...      0      1      1      0      0
4  20.17  5.625  1.71    0  120.0  ...      1      0      0      0      1

[5 rows x 46 columns]


array([[3.083e+01, 0.000e+00, 1.250e+00, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       [5.867e+01, 4.460e+00, 3.040e+00, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       [2.450e+01, 5.000e-01, 1.500e+00, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [2.525e+01, 1.350e+01, 2.000e+00, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       [1.792e+01, 2.050e-01, 4.000e-02, ..., 1.000e+00, 0.000e+00,
        0.000e+00],
       [3.500e+01, 3.375e+00, 8.290e+00, ..., 1.000e+00, 0.000e+00,
        0.000e+00]])

## Decision Tree with train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### max_depth = None

In [8]:
dtree = DecisionTreeClassifier(criterion='entropy', random_state=0)
dtree.fit(X_train, y_train)
predictions = dtree.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
accuracy

0.8115942028985508

In [0]:
tree.export_graphviz(dtree, out_file='dtree2.dot')

### max_depth = 5

In [10]:
dtree = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=0)
dtree.fit(X_train, y_train)
predictions = dtree.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
accuracy

0.8405797101449275

In [0]:
tree.export_graphviz(dtree, out_file='dtree2_depth5.dot')

## Decision Tree with K-Fold Cross Validation

### max_depth = 5

In [12]:
kf = KFold(n_splits=10)
kf.get_n_splits(X)

length_to_split = int(len(X) * 0.8)
# Splitting the X and y into train and test datasets
X_train, X_test = X[:length_to_split], X[length_to_split:]
y_train, y_test = y[:length_to_split], y[length_to_split:]
 
# Print the size of the train and test dataset
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

dtree = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=0)
dtree = dtree.fit(X_train, y_train)

predictions = dtree.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
accuracy

(552, 46) (138, 46)
(552,) (138,)


0.9057971014492754

In [0]:
tree.export_graphviz(dtree, out_file='dtree2_kfold.dot')